In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# !conda install pytorch torchvision torchaudio cudatoolkit=11.0 -c pytorch

In [ ]:
# from skimage import io, transform
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import cv2
import time
import os
import copy
from ipywidgets import FloatProgress


import os
from sklearn.model_selection import train_test_split
import tqdm.notebook as tqdm


import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils,models
from torchvision.utils import make_grid


import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
torch.__version__

In [ ]:
train_pet_meta_data = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/train.csv')
test_pet_meta_data = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/test.csv')
train_pet_meta_data

0. validate all the images in train folders has meta data
1. Create a pytorch dataset to load image and metadata
2. Split training data into train and validation set and use test for final evaluation
3. download pretrained densnet model 
4. modify the model to take meta data as input at after flattening 
5. add dense layers 


# EDA

In [ ]:
train_pet_meta_data.info()

In [ ]:
root_dir = "/kaggle/input/petfinder-pawpularity-score/"
plt.figure(figsize=(10,8))

for i, j in enumerate(np.random.randint(0, 100,size =6)):
    plt.subplot(2,3, i+1)
    sample_image = plt.imread( root_dir+ "train/" +train_pet_meta_data.Id[j] +".jpg")
    plt.imshow(sample_image)
plt.show()

In [ ]:
sample_image.shape

In [ ]:
class PawPularityDataset(Dataset):
    def __init__(self, csv_file, root_dir, transforms=None):
        self.paw_pularity_frame = csv_file
        self.root_dir = root_dir
        self.transforms = transforms
        self.is_test_dataset = True if "Pawpularity" not in self.paw_pularity_frame.columns else False
        
    def __len__(self):
        return len(self.paw_pularity_frame)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        image_path = os.path.join(self.root_dir,
                                self.paw_pularity_frame.iloc[idx, 0] + ".jpg")
        image = plt.imread(image_path)
        id = self.paw_pularity_frame.iloc[idx, 0]
        
        
        
        meta_data = np.array(self.paw_pularity_frame.iloc[idx, 1:13])
        meta_data = torch.tensor(meta_data.astype('float'), dtype=torch.float)
        meta_data = meta_data / 2 + 0.1 
        sample = {"image": image, 'meta_data': meta_data , "id":id }
        
        if self.is_test_dataset == False:
            score = self.paw_pularity_frame.iloc[idx, 13]
            score = torch.tensor(score, dtype=torch.float)
            score = score.unsqueeze(-1)
            sample['score'] = score 
        
        if self.transforms:
            sample['image'] = self.transforms(sample['image'])

        return sample


In [ ]:
train_images_data , val_images_data = train_test_split(train_pet_meta_data,train_size=0.8)
print(len(train_images_data), len(val_images_data))

train_transforms = transforms.Compose([
     transforms.ToPILImage(),
    transforms.Resize((224,224)),
    # transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transforms = transforms.Compose([
     transforms.ToPILImage(),
    transforms.Resize((224,224)),
    # transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

N_EPOCHS = 156
BATCH_SIZE =128

train_dataset = PawPularityDataset(train_images_data, root_dir+"train", transforms = train_transforms)
val_dataset = PawPularityDataset(val_images_data, root_dir+"train", transforms = val_transforms)
test_dataset = PawPularityDataset(test_pet_meta_data, root_dir+"test", transforms = val_transforms)

train_loader = DataLoader( train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader( val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_loader = DataLoader( test_dataset, batch_size=1, shuffle=False, num_workers=0)

In [ ]:
for item in test_loader:
    print(item.keys())
    print("image: ", item["image"].shape)
    print("meta_data: ", item["meta_data"].shape)
    print("id: ", len(item["id"]))
    try:
        print("score: ", item["score"].shape)
    except:
        print()
    break

In [ ]:
class ImageTabularDenseNet(nn.Module):
    def __init__(self, model):
        super(ImageTabularDenseNet, self).__init__()
        self.model_features = model
        
        for param in self.model_features.parameters():
            param.requires_grad = False
        self.model_features.classifier = nn.Sequential( 
            nn.Linear(1024, 1024), 
            nn.ReLU(),)
        
        self.model_classifier = nn.Sequential( 
                                    nn.Linear(1024 + 12, 1024),
                                    nn.BatchNorm1d(1024),
                                    nn.ReLU(),
                                    nn.Linear(1024, 512),
                                    nn.BatchNorm1d(512),
                                    nn.ReLU(),
                                    nn.Linear(512, 512),
                                    nn.BatchNorm1d(512),
                                    nn.ReLU(),
                                    nn.Linear(512, 1),
                                    nn.Sigmoid())
        
        for params in self.model_features.named_parameters():
            if "features.denseblock4.denselayer16" in params[0]: 
                # print(params[1].requires_grad)
                params[1].requires_grad = True
        
    def forward(self,images, meta_data):
        output = self.model_features(images)
        combined_data = torch.cat([output, meta_data], dim=1)
        output_1 = self.model_classifier(combined_data)
        return output_1 * 100



In [ ]:
model_ft = models.densenet121(pretrained=False)
image_tabular_net = ImageTabularDenseNet(model_ft)
# image_tabular_net

optimizer = torch.optim.AdamW(image_tabular_net.parameters(), lr=0.0001)
loss_function = torch.nn.MSELoss(reduction="mean")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_model(model):
    best_metric = 9999
    best_metric_epoch = -1
    epoch_loss_values = []
    metric_values = []
    root_dir =''
    
    val_interval = 2
    model = model.to(device)
    for epoch in range(N_EPOCHS):
        print("-" * 10)
        print(f"epoch {epoch + 1}/{N_EPOCHS}")
        model.train()
        epoch_loss = 0
        step = 0
        for batch_data in train_loader:
            step += 1
            images,meta_data, labels = batch_data["image"].to(device), batch_data["meta_data"].to(device), batch_data["score"].to(device)
            optimizer.zero_grad()
            outputs = model(images, meta_data)
            loss = loss_function(outputs, labels)
            loss = torch.sqrt(loss)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            if step % 5 == 0 : 
                print(
                    f"{step}/{len(train_dataset) // train_loader.batch_size}, "
                    f"train_loss: {loss.item():.4f}")
            epoch_len = len(train_dataset) // train_loader.batch_size
        epoch_loss /= step
        epoch_loss_values.append(epoch_loss)
        
        print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

        if (epoch + 1) % val_interval == 0:
            model.eval()
            with torch.no_grad():
                y_pred = torch.tensor([], dtype=torch.float32, device=device)
                y = torch.tensor([], dtype=torch.long, device=device)
                for val_data in val_loader:
                    val_images,val_meta_data, val_labels = (
                        val_data["image"].to(device),
                        val_data["meta_data"].to(device),
                        val_data["score"].to(device),
                    )
                    y_pred = torch.cat([y_pred, model(val_images,val_meta_data)], dim=0)
                    y = torch.cat([y, val_labels], dim=0)
                val_loss = loss_function(y_pred, y)
                val_loss = torch.sqrt(val_loss)

                metric_values.append(val_loss)

                if val_loss < best_metric:
                    best_metric = val_loss
                    best_metric_epoch = epoch + 1
                    torch.save(model.state_dict(), os.path.join(
                        root_dir, "best_metric_model.pth"))
                    print("saved new best metric model")
                print(
                    f"current epoch: {epoch + 1} current val loss: {val_loss:.4f}"
                    f" val loss: {best_metric:.4f}"
                    f" at epoch: {best_metric_epoch}"
                )

    print(
        f"train completed, best_metric: {best_metric:.4f} "
        f"at epoch: {best_metric_epoch}")
    return model

In [ ]:
# image_tabular_net = train_model(image_tabular_net)

In [ ]:
image_tabular_net.load_state_dict(torch.load("../input/pawpularity-best-model/best_metric_model.pth"))
image_tabular_net.to(device).eval()
ids = []
predictions = []
with torch.no_grad():
    for test_data in test_loader:
        test_images,test_meta_data, test_id = (
            test_data["image"].to(device),
            test_data["meta_data"].to(device),
            test_data["id"],
        )
        y_pred =  image_tabular_net(test_images,test_meta_data)
        
        for i in range(y_pred.shape[0]):
            predictions.append(y_pred[i].item())
            ids.append(test_id[i])

results_df = pd.DataFrame({"id":ids, "predictions": predictions})

results_df.to_csv("submission.csv", index=False)
    